In [1]:
%matplotlib inline
import collections 
import numpy as np
import scipy as sp
import pandas as pd
import time
from datetime import date, timedelta
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import re
import urllib2
#import regex
#import locale
#import usaddress
#import geograpy
#import nltk
from pyquery import PyQuery as pq
from bs4 import BeautifulSoup
import requests

In [14]:
climb_req=requests.get("http://www.mountainproject.com/v/okelleys-crack/105722353")
climb_=pq(climb_req.text)
climb_loc = climb_("#navBox").find('a')
locationlist = []
for i in range(0, len(climb_loc)):
    locationlist.append(climb_loc.eq(i).attr('href'))
locationlist

['/destinations/',
 '/v/california/105708959',
 '/v/joshua-tree-national-park/105720495',
 '/v/echo-rock-area/105720576',
 '/v/rusty-wall/105720813']

In [2]:
# global data storage
climbsjson = collections.defaultdict(dict) # double nested dict
usersjson = collections.defaultdict(dict) # double nested dict
ucjson = collections.defaultdict(dict) #double nested dict

In [3]:
# sub-functions and initializations for user/climb_scrape_func-s

def parse_climb_link(href_link):
    return href_link.split('/')[-1], href_link.split('/')[-2]

def parse_user_link(href_link):
    return href_link.split('/')[-1], href_link.split('/')[-3]

# 16. get star info from td pairs for climb parser
def star_quality_func(td1, td2, climb_id):
    user_id, user_name = parse_user_link(td1('a').attr('href'))
    rating = int(re.search(r'starsHtml\(([0-5]),', 
                       td2('script').text()).group(1)) - 1
    ucjson[climb_id + "_" + user_id]["user_id"] = user_id
    ucjson[climb_id + "_" + user_id]["climb_id"] = climb_id
    ucjson[climb_id + "_" + user_id]["star_rating"] = rating

# 19. get sugg_rate info from td pairs
def sugg_rating_func(td1, td2, climb_id):
    user_id, user_name = parse_user_link(td1('a').attr('href'))
    ucjson[climb_id + "_" + user_id]["user_id"] = user_id
    ucjson[climb_id + "_" + user_id]["climb_id"] = climb_id
    
    for key, regex in diff_regexes.iteritems():
        temp = re.search(regex, td2.text())
        if  temp != None:
            ucjson[climb_id + "_" + user_id]["sugg_" + key] = temp.group()
        else:
            ucjson[climb_id + "_" + user_id]["sugg_" + key] = 0

aid_re = r'(A[0-5])|(C[0-5])[-,+]?'
boulder_re = r'V\d\d?[-,+]?'
ice_re = r'WI[1-6][+,-]?'
mixed_re = r'M\d\d?[+,-]?'
rock_re = r'5.\d\d?(([a-d]/[a-d])|([a-d]))?[+,-]?'

diff_regexes = {"aid_grade": aid_re, "boulder_grade": boulder_re, 
               "ice_grade": ice_re, "mixed_grade": mixed_re, 
               "rock_grade": rock_re}

user_type_regexes = {"Trad": rock_re, "Sport": rock_re, 
                     "Boulders": boulder_re, "Aid": aid_re, 
                     "Ice": ice_re, "Mixed": mixed_re}

typesdict = {"trad": 0, "alpine": 0, "ice": 0, "sport": 0,
        "boulder": 0, "aid": 0, "mixed": 0, "TR,": 0}



In [4]:
def user_scrape_func(userlink):
    
    # initialize local user dict to be appended to usersjson at end
    userdict = {"user_link": userlink, "user_id": 0, "user_name": "undef", "location_place": "undef", 
                "location_state": "undef", "age": 0, "gender": "undef", 
                "member_date": 0}
    
    # 1. get user_name and id-s
    user_id, user_name = parse_user_link(userlink)
    userdict["user_id"], userdict["user_name"] = user_id, user_name
    
    if len(usersjson[user_id]) != 0:
        return
    
    # get page and convert to pyquery obj
    user_req=requests.get(userlink)
    user_=pq(user_req.text)
    
    # extract personal data tag
    user_main = user_('div.personalData')
    user_pers = user_main("div:contains('Personal:')")('em')
    
    # 2. if exists, get place of residence
    if re.search('Lives in', user_pers.text()) != None:
        userdict["location_place"] = user_pers.text().split(',')[0].replace('Lives in ',"")
        
        # check for state separated by comma (will need to clean this)
        maybe_state = user_pers.text().split(',')[1]
        if (re.search(r'(years old)|(male)|(female)', maybe_state) == None):
            userdict["location_state"] = maybe_state
    
    # 3. get age
    temp_age = re.search(r'(\d\d?) years old', user_pers.text())
    if temp_age:
        userdict["age"] = int(temp_age.group(1))
    
    # 4. get gender
    temp_gender = re.search(r'(male)|(female)', user_pers.text(), re.I)
    if temp_gender:
        userdict["gender"] = temp_gender.group()
    
    # 5. get liked types and difficulties
    user_diffs = user_main('table')

    for key, value in user_type_regexes.items():
        if key != "Boulders":
            tdwithkey = user_diffs.find("tr").children("td:contains('{}')".format(key))
            if tdwithkey:
                userdict["climbs_" + str.lower(key)] = 1
                leadhtml = tdwithkey.next().html()
                followhtml = tdwithkey.next().next().html()  
                templeaddiff = re.search("{}".format(value), leadhtml, re.I)
                tempfollowdiff = re.search("{}".format(value), followhtml, re.I)
                if templeaddiff:
                    userdict["lead_diff_" + str.lower(key)] = templeaddiff.group() 
                else:
                    userdict["lead_diff_" + str.lower(key)] = '0'
                if tempfollowdiff:
                    userdict["follow_diff_" + str.lower(key)] = tempfollowdiff.group()
                else:
                    userdict["follow_diff_" + str.lower(key)] = '0'
            else:
                userdict["climbs_" + str.lower(key)] = 0

        elif key == "Boulders":
            boulderhtml = user_diffs.find("tr").children("td:contains('{}')".format(key))
            if boulderhtml:
                userdict["climbs_boulder"] = 1
                boulderdiff = boulderhtml.next().html()
                tempboulderdiff = re.search("{}".format(value), boulderdiff)
                if tempboulderdiff:
                    userdict["boulder_diff"] = tempboulderdiff.group()
            else:
                userdict["climbs_boulder"] = 0
            
    
    # 6. get date 
    user_left_box = user_("td[width='190']")('b')
    if user_left_box:
        userdict["member_date"] = time.strptime(user_left_box.eq(0).text(), 
                                                "%b %d, %Y")
    
    
    
    # might be good to add a bgcolor attribute ref to find only populated trs
    #### 7. get stars
    starsmain = "http://www.mountainproject.com/u/{}?action=contribs&what=SCORE&".format(user_id)
    stars_main_req=requests.get(starsmain)
    stars_main_page = pq(stars_main_req.text)
    
    # if multiple pages
    num_pages = stars_main_page("h1").next().next().find('tr').eq(1).text()
    if num_pages:
        pages = int(re.search(r'Page \d\d? of (\d\d?)', num_pages).group(1))
    else:
        pages = 1
    
    # parse all pages
    for page in range(1, pages + 1):
        time.sleep(1)
        starslink = "http://www.mountainproject.com/u/{}?action=contribs&what=SCORE&&page={}".format(user_id, page)
        stars_req=requests.get(starslink)
        stars_page = pq(stars_req.text)
        star_rows = stars_page("table[width*='100%'][border='0'][class='objectList']").eq(1).find('tr')

        for i in range(2, len(star_rows)):
            tds = star_rows.eq(i).find('td')
            # get user and climb id-s
            climb_link = tds.eq(0).find('a').attr('href')
            climb_id, _ = parse_climb_link(climb_link)
            ucjson[climb_id + "_" + user_id]["user_id"] = user_id
            ucjson[climb_id + "_" + user_id]["climb_id"] = climb_id
            #get rating
            rating = int(re.search(r'starsHtml\(([0-5]),', 
                           tds.eq(1)('script').text()).group(1)) - 1   
            ucjson[climb_id + "_" + user_id]["star_rating"] = rating

    # 8. ticks
    ticksmain = "http://www.mountainproject.com/u/{}?action=ticks&".format(user_id)
    ticks_main_req=requests.get(ticksmain)
    ticks_main_page = pq(ticks_main_req.text)
    dates_wrong = 0
    
    # if multiple pages
    num_pages = ticks_main_page("table#stats").next().next().find('tr').eq(0).text()
    if num_pages:
        pages = int(re.search(r'Page \d\d? of (\d\d?)', num_pages).group(1))
    else:
        pages = 1
    
    # parse all pages
    for page in range(1, pages + 1):
        time.sleep(1)
        tickslink = "http://www.mountainproject.com/u/{}?action=ticks&&page={}".format(user_id, page)
        ticks_req=requests.get(tickslink)
        ticks_page = pq(ticks_req.text)
        tick_rows = ticks_page("table[width*='100%'][border='0'][class='objectList']").eq(1).find('tr')

        # parse all rows
        for i in range(2, len(tick_rows)):
            tds = tick_rows.eq(i).find('td')
            
            # get climb id, user id and mark ticked
            climb_link = tds.eq(0).find('a').attr('href')
            climb_id, _ = parse_climb_link(climb_link)
            ucjson[climb_id + "_" + user_id]["user_id"] = user_id
            ucjson[climb_id + "_" + user_id]["climb_id"] = climb_id
            ucjson[climb_id + "_" + user_id]["ticked"] = 1

            # get date
            dates = None
            days_ago = re.search(r'(\d\d?) days ago', tds.eq(4)('p').text())
            if days_ago:
                days_num = int(days_ago.group(1))
                dates = date.today() - timedelta(days=days_num)
                ucjson[climb_id + "_" + user_id]["ticked_date"] = dates
            else:
                # exception if for some dates with year -0001
                try:
                    dates = time.strptime(tds.eq(4)('p').text(), "%b %d, %Y")
                    ucjson[climb_id + "_" + user_id]["ticked_date"] = dates
                except ValueError:
                    dates_wrong += 1
    
    # add userdict to usersjson
    usersjson[user_id] = userdict

In [5]:
def climb_scrape_func(climb_link):
    
    # initialize local climbdict
    climbdict = {"climb_id": 0, "climb_name": 'undef',
             "avg_stars": 0, "pitches": 0, "feet": 0, "climb_link": climb_link}
    
    # 1. get keys
    climb_id, climb_name = parse_climb_link(climb_link)
    climbdict["climb_id"] = climb_id
    climbdict["climb_name"] = climb_name
    
    # climb already scraped, then abort
    if len(climbsjson[climb_id]) != 0:
        return
    
    # get page into pyquery object
    climb_req=requests.get(climb_link)
    climb_=pq(climb_req.text)
    
    # 2. separate climb div
    climb_div = climb_('div#rspCol800')
    
    # 3. get climb summary div
    climb_summstats = climb_div('div.rspCol[style="max-width:500px;"]')
    
    # 4. get grade ratings
    climb_summstats_subtitle = climb_summstats('h3')
    
    for key, regex in diff_regexes.items():
        temp = re.search(regex, climb_summstats_subtitle.text())
        if  temp != None:
            climbdict["guide_" + key] = temp.group()
        else:
            climbdict["guide_" + key] = 0
            
    # 5. get average star rating
    average_stars_script_html = climb_summstats('span#routeStars')('script').html()
    average_stars_regexobj = re.search(r'[1-5]\.\d\d?\d?\d?', 
                                       average_stars_script_html)
    if average_stars_regexobj != None:     
        avg_stars = float(average_stars_regexobj.group()) - 1
        climbdict["avg_stars"] = avg_stars
    else:
        avg_stars = 0
        
    # 6. get rows of table
    climb_summstats_table = climb_summstats('table')('tr')
    
    # 7. get climb types
    for key in typesdict:
        if re.search(r'{}'.format(key), 
                    climb_summstats_table.eq(0).text(), 
                    flags = re.I) != None:
            climbdict["type_" + key] = 1
    
    # 8. get the number of pitches
    pitches_regobj = re.search(r'(\d\d?) pitches', 
                    climb_summstats_table.eq(0).text(), 
                    flags = re.I)
    if pitches_regobj:
        climbdict["pitches"] = int(pitches_regobj.group(1))
        
    # 9. get number of feet
    feet_regobj = re.search(r" (\d\d?\d?\d?\d?)'", 
                    climb_summstats_table.eq(0).text(), 
                    flags = re.I)
    if feet_regobj:
        climbdict["feet"] = int(feet_regobj.group(1))

    # 10. get number of feet
    grade_regobj = re.search(r'Grade ([I,II,III,IV,V,VI])', 
                    climb_summstats_table.eq(0).text(), 
                    flags = re.I)
    if grade_regobj:
        climbdict["grade"] = grade_regobj.group(1)
        
    # 11. get concensus rating
    for key, regex in diff_regexes.iteritems():
        temp = re.search(regex,climb_summstats_table.eq(1).text())
        if  temp != None:
            climbdict["concensus_" + key] = temp.group()
        else:
            climbdict["concensus_" + key] = 0
   
    # 12. get FA year
    fa_regobj = re.search(r"[18,19,20]\d\d\d",climb_summstats_table.eq(2).text())
    if fa_regobj:
        climbdict["fa_year"] = int(fa_regobj.group())
    
    # 13. get page views
    views_regobj = re.search(r"\d?\d?\d?,?\d?\d\d",climb_summstats_table.eq(3).text())
    if views_regobj:
        climbdict["page_views"] = int(views_regobj.group().replace(',',''))
        
    # 14. get detailed users-climb page
    stats_link = ("http://mountainproject.com/scripts/ShowObjectStats.php?id=%s"
                  % climbdict['climb_id'])
    climb_stats_req=requests.get(stats_link)
    stats_=pq(climb_stats_req.text)

    # 15. get star quality votes tables
    stats_stars_table = stats_("span:contains('Star Quality Votes')").next().next().find('tr')  
    
    # 17. iterate through stars table and populate ucjson 
    for i in range(0, len(stats_stars_table)):
        tds = stats_stars_table.eq(i).find('td')
        if len(tds) > 0:
            star_quality_func(tds.eq(0),tds.eq(1), climb_id)
            if len(tds) == 4:
                star_quality_func(tds.eq(2),tds.eq(3), climb_id)
                
    # 18. get suggested ratings rows
    stats_sugg_table = stats_("span:contains('Suggested Ratings')").next().next().find('tr')  
    
    # 20. iterate through suggested ratings table and populate ucjson 
    for i in range(0, len(stats_sugg_table)):
        tds = stats_sugg_table.eq(i).find('td')
        if len(tds) > 1:
            sugg_rating_func(tds.eq(0),tds.eq(1), climb_id)
            if len(tds) == 4:
                sugg_rating_func(tds.eq(2),tds.eq(3), climb_id)
                
    # 21. 
    stats_ticks_table = stats_("span:contains('Ticks')").next().next().find('tr')  

    # 22. iterate through ticks ratings table and populate ucjson 
    # this might require a tweek if dates are ever listed as ("\d\d? days ago")
    dates_wrong = 0
    for i in range(0, len(stats_ticks_table)):
        tds = stats_ticks_table.eq(i).find('td')
        if len(tds) == 3:
            user_id, user_name = parse_user_link(tds.eq(0)('a').attr('href'))
            ucjson[climb_id + "_" + user_id]["user_id"] = user_id
            ucjson[climb_id + "_" + user_id]["climb_id"] = climb_id
            try: 
                date = time.strptime(tds.eq(1).text(), "%b %d, %Y")
                ucjson[climb_id + "_" + user_id]["tick_date"] = date
            except ValueError:
                dates_wrong += 1
                
    # add climbdict to climbsjson
    climbsjson[climb_id] = climbdict

Feel free to play with testing below, it will be important that we understand what the data we have later means:

In [6]:
user_scrape_func("http://www.mountainproject.com/u/rajko-radovanovic//108972429")

In [7]:
usersjson

defaultdict(dict,
            {'108972429': {'age': 22,
              'climbs_aid': 0,
              'climbs_boulder': 0,
              'climbs_ice': 1,
              'climbs_mixed': 1,
              'climbs_sport': 1,
              'climbs_trad': 1,
              'follow_diff_ice': 'WI5',
              'follow_diff_mixed': 'M5',
              'follow_diff_sport': u'5.10a',
              'follow_diff_trad': u'5.9',
              'gender': 'Male',
              'lead_diff_ice': u'WI3',
              'lead_diff_mixed': u'M1',
              'lead_diff_sport': u'5.9',
              'lead_diff_trad': u'5.6',
              'location_place': 'Cambridge',
              'location_state': ' MA',
              'member_date': time.struct_time(tm_year=2014, tm_mon=5, tm_mday=13, tm_hour=0, tm_min=0, tm_sec=0, tm_wday=1, tm_yday=133, tm_isdst=-1),
              'user_id': '108972429',
              'user_link': 'http://www.mountainproject.com/u/rajko-radovanovic//108972429',
              'user_name

In [8]:
userlinks_test = ["http://www.mountainproject.com/u/claire-stolz//108141508",
                  "http://www.mountainproject.com/u/nick-grant//107851075", 
                  "http://www.mountainproject.com/u/aaron-carney//110105933",
                  'http://www.mountainproject.com/u/aaron-conway//108046170',
                  "http://www.mountainproject.com/u/alan-margolies//108119044",
                  "http://www.mountainproject.com/u/amelia-litz//108635482",
                  "http://www.mountainproject.com/u/aaron-chambers//108225696", 
                  "http://www.mountainproject.com/u/richardo//107499279", 
                  "http://www.mountainproject.com/u/ray-shader//106642411", 
                  "http://www.mountainproject.com/u/rob-albert//106146571", 
                  "http://www.mountainproject.com/u/robbie-mize//107316636", 
                  "http://www.mountainproject.com/u/ross-keller//11047", 
                  "http://www.mountainproject.com/u/ross-merridock//108379018"]

In [9]:
%%time
for userlink in userlinks_test:
    user_scrape_func(userlink)
    time.sleep(1)

Wall time: 1min 41s


In [10]:
usersjson

defaultdict(dict,
            {'106146571': {'age': 0,
              'climbs_aid': 0,
              'climbs_boulder': 0,
              'climbs_ice': 0,
              'climbs_mixed': 0,
              'climbs_sport': 0,
              'climbs_trad': 0,
              'gender': 'undef',
              'location_place': 'undef',
              'location_state': 'undef',
              'member_date': time.struct_time(tm_year=2008, tm_mon=4, tm_mday=22, tm_hour=0, tm_min=0, tm_sec=0, tm_wday=1, tm_yday=113, tm_isdst=-1),
              'user_id': '106146571',
              'user_link': 'http://www.mountainproject.com/u/rob-albert//106146571',
              'user_name': 'rob-albert'},
             '106642411': {'age': 0,
              'climbs_aid': 0,
              'climbs_boulder': 0,
              'climbs_ice': 0,
              'climbs_mixed': 0,
              'climbs_sport': 0,
              'climbs_trad': 0,
              'gender': 'undef',
              'location_place': 'undef',
             

In [11]:
climblinks_test = ["http://www.mountainproject.com/v/high-exposure/105798994", 
                   "http://www.mountainproject.com/v/directissima/105799563",
                   "http://www.mountainproject.com/v/enduro-mans-longest-hangout/107328856", 
                   "http://www.mountainproject.com/v/ridicullissima/105888125", 
                   "http://www.mountainproject.com/v/directississima/105799077", 
                   "http://www.mountainproject.com/v/hammond-eggs/107315322", 
                   "http://www.mountainproject.com/v/kelloggs/105908309", 
                   "http://www.mountainproject.com/v/pinnacle-gully/105890658", 
                   "http://www.mountainproject.com/v/diagonal-gully/106707853", 
                   "http://www.mountainproject.com/v/el-cap-tree/107057624", 
                   "http://www.mountainproject.com/v/quo-vadis/106683500", 
                   "http://www.mountainproject.com/v/dawn-wall-free/109951912", 
                   "https://www.mountainproject.com/v/shoestring-gully/106100769"]


In [12]:
%%time 
for climblink in climblinks_test:
    climb_scrape_func(climblink)
    time.sleep(1)

Wall time: 21.9 s


In [13]:
print "Climbs: ", len(climbsjson)
for key, value in climbsjson.items():
    print "Climbinfo: " + key + ": ", len(value)

print "User_climb_combos: ", len(ucjson)
for key, value in ucjson.items():
    print "user_climb_data: " + key + ": ", len(value)

Climbs:  13
Climbinfo: 109951912:  19
Climbinfo: 105908309:  20
Climbinfo: 105890658:  21
Climbinfo: 107057624:  20
Climbinfo: 106683500:  20
Climbinfo: 107328856:  20
Climbinfo: 105799563:  19
Climbinfo: 106707853:  19
Climbinfo: 105798994:  19
Climbinfo: 107315322:  18
Climbinfo: 105888125:  19
Climbinfo: 105799077:  19
Climbinfo: 106100769:  19
User_climb_combos:  2808
user_climb_data: 105798994_106253664:  3
user_climb_data: 105799563_107989123:  3
user_climb_data: 105799563_108724264:  3
user_climb_data: 105798994_108552179:  3
user_climb_data: 105799563_106344939:  3
user_climb_data: 105798994_106897206:  4
user_climb_data: 105798994_110534118:  4
user_climb_data: 105799563_107518458:  4
user_climb_data: 105798994_111007009:  3
user_climb_data: 105896493_107851075:  3
user_climb_data: 105798994_106347093:  3
user_climb_data: 105798994_106231351:  3
user_climb_data: 105717562_11047:  3
user_climb_data: 105799563_107002528:  3
user_climb_data: 105798994_107532062:  9
user_climb_dat

In [14]:
climbsjson["107057624"]

{'avg_stars': 2.7143,
 'climb_id': '107057624',
 'climb_link': 'http://www.mountainproject.com/v/el-cap-tree/107057624',
 'climb_name': 'el-cap-tree',
 'concensus_aid_grade': u'C1+',
 'concensus_boulder_grade': 0,
 'concensus_ice_grade': 0,
 'concensus_mixed_grade': 0,
 'concensus_rock_grade': u'5.7',
 'fa_year': 1952,
 'feet': 400,
 'grade': u'I',
 'guide_aid_grade': u'C2',
 'guide_boulder_grade': 0,
 'guide_ice_grade': 0,
 'guide_mixed_grade': 0,
 'guide_rock_grade': u'5.7',
 'pitches': 5,
 'type_aid': 1,
 'type_trad': 1}

In [15]:
usersjson["107499279"]

{'age': 27,
 'boulder_diff': u'V6',
 'climbs_aid': 0,
 'climbs_boulder': 1,
 'climbs_ice': 0,
 'climbs_mixed': 0,
 'climbs_sport': 1,
 'climbs_trad': 1,
 'follow_diff_sport': u'5.12c',
 'follow_diff_trad': u'5.12b',
 'gender': 'Male',
 'lead_diff_sport': u'5.12b',
 'lead_diff_trad': u'5.12a',
 'location_place': 'Culver City',
 'location_state': ' CA 90232',
 'member_date': time.struct_time(tm_year=2012, tm_mon=3, tm_mday=5, tm_hour=0, tm_min=0, tm_sec=0, tm_wday=0, tm_yday=65, tm_isdst=-1),
 'user_id': '107499279',
 'user_link': 'http://www.mountainproject.com/u/richardo//107499279',
 'user_name': 'richardo'}

In [16]:
usersjson

defaultdict(dict,
            {'106146571': {'age': 0,
              'climbs_aid': 0,
              'climbs_boulder': 0,
              'climbs_ice': 0,
              'climbs_mixed': 0,
              'climbs_sport': 0,
              'climbs_trad': 0,
              'gender': 'undef',
              'location_place': 'undef',
              'location_state': 'undef',
              'member_date': time.struct_time(tm_year=2008, tm_mon=4, tm_mday=22, tm_hour=0, tm_min=0, tm_sec=0, tm_wday=1, tm_yday=113, tm_isdst=-1),
              'user_id': '106146571',
              'user_link': 'http://www.mountainproject.com/u/rob-albert//106146571',
              'user_name': 'rob-albert'},
             '106642411': {'age': 0,
              'climbs_aid': 0,
              'climbs_boulder': 0,
              'climbs_ice': 0,
              'climbs_mixed': 0,
              'climbs_sport': 0,
              'climbs_trad': 0,
              'gender': 'undef',
              'location_place': 'undef',
             

In [ ]:
for climb_user_concat in ucjson.keys():
    for feature, value in ucjson[key].items():
        dataframe[climb_user_concat (indexing row not column)][feature (indexing column)] = value
    for feature, value in usersjson[ucjson[climb_user_concat]["user_id"]]:
        dataframe[climb_user_concat (indexing row not column)][feature (indexing column)] = value
    for feature, value in climbsjson[ucjson[climb_user_concat]["climb_id"]]:
        dataframe[climb_user_concat (indexing row not column)][feature (indexing column)] = value
